In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd '/content/drive/MyDrive/NYU_class/NLU/Project'

/content/drive/MyDrive/NYU_class/NLU/Project


In [3]:
!pip install transformers

     |████████████████████████████████| 2.2MB 5.1MB/s 
     |████████████████████████████████| 870kB 18.7MB/s 
     |████████████████████████████████| 3.3MB 22.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=1a476d2c3f4e9f767c23963c76b04cb79ecb8ec51f23ef26280605c5577723f5
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sacremoses


In [4]:
import torch
import torch.nn as nn 
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset

from transformers import AutoTokenizer, BertTokenizer, EvalPrediction, BertPreTrainedModel, BertConfig, BertModel, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup

import random
import pandas as pd
import numpy as np
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

## Data pre-processing

Strict-matched

In [5]:
circa_og = pd.read_csv('circa-data.tsv', sep='\t', index_col='id')
circa_s = circa_og.drop(circa_og.loc[circa_og['goldstandard1']=='Other'].index)
circa_s = circa_s.drop(circa_s.loc[circa_s['goldstandard1'].isnull()].index)
circa_s = circa_s.drop(circa_s.loc[circa_s['goldstandard1']=='I am not sure how X will interpret Y’s answer'].index)

In [6]:
YN_s = (circa_s['question-X'].map(str)+' '+circa_s['answer-Y']).apply(lambda row: row.strip())
strict_labels = circa_s['goldstandard1'].unique()
strict_label = circa_s['goldstandard1']
strict_dict = {}
for idx, label in enumerate(strict_labels):
    strict_dict[label] = idx
circa_s['strict'] = circa_s.goldstandard1.replace(strict_dict)
strict = circa_s['strict']

Relaxed-matched

In [7]:
circa_r = circa_og.drop(circa_og.loc[circa_og['goldstandard2']=='Other'].index)
circa_r = circa_r.drop(circa_r.loc[circa_r['goldstandard2'].isnull()].index)
relaxed = circa_r['goldstandard2']
YN_r = (circa_r['question-X'].map(str)+' '+circa_r['answer-Y']).apply(lambda row: row.strip())

In [8]:
relaxed_labels = circa_r['goldstandard2'].unique()
relaxed_dict = {}
for idx, label in enumerate(relaxed_labels):
    relaxed_dict[label] = idx
relaxed_dict

{'In the middle, neither yes nor no': 2,
 'No': 1,
 'Yes': 0,
 'Yes, subject to some conditions': 3}

In [9]:
circa_r['relaxed'] = circa_r.goldstandard2.replace(relaxed_dict)
relaxed = circa_r['relaxed']

In [10]:
train_relaxed, val_relaxed, trainy_relaxed, valy_relaxed = train_test_split(YN_r, relaxed, test_size=.4, stratify=relaxed)
test_relaxed, dev_relaxed, testy_relaxed, devy_relaxed = train_test_split(val_relaxed, valy_relaxed, test_size=.5, stratify=valy_relaxed)

## Modeling

In [11]:
import tensorflow as tf

# Get the GPU device name.
device_name = tf.test.gpu_device_name()
print(device_name)

if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

/device:GPU:0


### Strict

In [12]:
model_name = 'bert-large-uncased'
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=len(strict_dict),output_attentions = False, output_hidden_states = False,)
tokenizer = BertTokenizer.from_pretrained(model_name, do_lower_case=True)

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint a

In [13]:
max_len = 0
for entry in YN_s.values:
    input_ids = tokenizer.encode(entry,  add_special_tokens=True)
    max_len = max(max_len, len(input_ids))
print(max_len)

43


In [14]:
strict_labels = circa_s['goldstandard1'].unique()
strict_label = circa_s['goldstandard1']
strict_dict = {}
for idx, label in enumerate(strict_labels):
    strict_dict[label] = idx
circa_s['strict'] = circa_s.goldstandard1.replace(strict_dict)
strict = circa_s['strict']

YN_s = (circa_s['question-X'].map(str)+' '+circa_s['answer-Y']).apply(lambda row: row.strip())

In [15]:
df = pd.concat([YN_s, strict_label, strict], axis=1).rename(columns={0:'YN_s'})
df

,YN_s,goldstandard1,strict
id,,,
0,Are you employed? I'm a veterinary technician.,Yes,0
1,Are you a fan of Korean food? I wouldn't say so,No,1
2,Are you bringing any pets into the flat? I do ...,No,1
3,Would you like to get some fresh air in your f...,Yes,0
4,Is your family still living in the neighborhoo...,"In the middle, neither yes nor no",2
...,...,...,...
34263,Do you like to drink? I am in AA.,No,1
34264,Do you like pie? My favorite pie is pecan.,Yes,0
34265,Want to go to a concert with me? I'd rather do...,No,1


In [16]:
train_strict, val_strict, trainy_strict, valy_strict = train_test_split(df.index.values, df.strict.values, test_size=.4, stratify=df.strict.values)
test_strict, dev_strict, testy_strict, devy_strict = train_test_split(val_strict, valy_strict, test_size=.5, stratify=valy_strict)

In [17]:
df['data_type'] = ['not_set']*df.shape[0]
df.loc[train_strict,'data_type'] = 'train'
df.loc[dev_strict,'data_type'] = 'dev'
df.loc[test_strict,'data_type'] = 'test'

In [18]:
df.groupby(['goldstandard1','strict','data_type']).count()

YN_s
goldstandard1                     strict data_type      
In the middle, neither yes nor no 2      dev         128
                                         test        127
                                         train       383
No                                1      dev        2166
                                         test       2166
                                         train      6497
Probably no                       4      dev         232
                                         test        232
                                         train       696
Probably yes / sometimes yes      3      dev         249
                                         test        249
                                         train       746
Yes                               0      dev        2901
                                         test       2901
                                         train      8702
Yes, subject to some conditions   5      dev         516
                                         test        517
                                         train      1550

In [19]:
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].YN_s.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)

encoded_data_dev = tokenizer.batch_encode_plus(
    df[df.data_type=='dev'].YN_s.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)


input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].strict.values)

input_ids_dev = encoded_data_dev['input_ids']
attention_masks_dev = encoded_data_dev['attention_mask']
labels_dev = torch.tensor(df[df.data_type=='dev'].strict.values)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2079: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [20]:
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_dev = TensorDataset(input_ids_dev, attention_masks_dev, labels_dev)

In [21]:
len(dataset_train)

18574

In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(strict_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [22]:
batch_size = 32

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_dev, 
                                   sampler=SequentialSampler(dataset_dev), 
                                   batch_size=batch_size)

optimizer = AdamW(model.parameters(),
                  lr = 3e-5,
                  eps = 1e-8)

In [23]:
epochs = 3
total_steps = len(dataloader_train) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, 
                                            num_training_steps = total_steps)

In [24]:
def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in strict_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [25]:
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [26]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

cuda


In [27]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [ ]:
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    torch.save(model.state_dict(), f'finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    dev_loss, predictions, true_vals = evaluate(dataloader_validation)
    dev_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {dev_loss}')
    tqdm.write(f'F1 Score (Weighted): {dev_f1}')


  0%|          | 0/3 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/581 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/581 [00:01<?, ?it/s, training_loss=0.161]

Epoch 1:   0%|          | 1/581 [00:01<13:44,  1.42s/it, training_loss=0.161]

Epoch 1:   0%|          | 1/581 [00:02<13:44,  1.42s/it, training_loss=0.157]

Epoch 1:   0%|          | 2/581 [00:02<13:39,  1.41s/it, training_loss=0.157]

Epoch 1:   0%|          | 2/581 [00:04<13:39,  1.41s/it, training_loss=0.174]

Epoch 1:   1%|          | 3/581 [00:04<13:38,  1.42s/it, training_loss=0.174]

Epoch 1:   1%|          | 3/581 [00:05<13:38,  1.42s/it, training_loss=0.123]

Epoch 1:   1%|          | 4/581 [00:05<13:37,  1.42s/it, training_loss=0.123]

Epoch 1:   1%|          | 4/581 [00:07<13:37,  1.42s/it, training_loss=0.281]

Epoch 1:   1%|          | 5/581 [00:07<13:37,  1.42s/it, training_loss=0.281]

Epoch 1:   1%|          | 5/581 [00:08<13:37,  1.42s/it, training_loss=0.194]

Epoch 1:   1%|          | 6/581 [00:08<13:38,  1.


Epoch 1
Training loss: 0.4647827268980456



  0%|          | 0/3 [30:52<?, ?it/s]

 33%|███▎      | 1/3 [15:46<31:32, 946.01s/it]

Epoch 2:   0%|          | 0/581 [00:00<?, ?it/s]

Validation loss: 0.5376409750777421
F1 Score (Weighted): 0.8153398686721676




Epoch 2:   0%|          | 0/581 [00:01<?, ?it/s, training_loss=0.139]

Epoch 2:   0%|          | 1/581 [00:01<13:53,  1.44s/it, training_loss=0.139]

Epoch 2:   0%|          | 1/581 [00:02<13:53,  1.44s/it, training_loss=0.187]

Epoch 2:   0%|          | 2/581 [00:02<13:54,  1.44s/it, training_loss=0.187]

Epoch 2:   0%|          | 2/581 [00:04<13:54,  1.44s/it, training_loss=0.118]

Epoch 2:   1%|          | 3/581 [00:04<13:55,  1.44s/it, training_loss=0.118]

Epoch 2:   1%|          | 3/581 [00:05<13:55,  1.44s/it, training_loss=0.137]

Epoch 2:   1%|          | 4/581 [00:05<13:53,  1.44s/it, training_loss=0.137]

Epoch 2:   1%|          | 4/581 [00:07<13:53,  1.44s/it, training_loss=0.170]

Epoch 2:   1%|          | 5/581 [00:07<13:51,  1.44s/it, training_loss=0.170]

Epoch 2:   1%|          | 5/581 [00:08<13:51,  1.44s/it, training_loss=0.078]

Epoch 2:   1%|          | 6/581 [00:08<13:51,  1.45s/it, training_loss=0.078]

Epoch 2:   1%|          | 6/581 [00:10<13:51,  1.45s/it, t

...

Validation loss: 0.5755915859925378
F1 Score (Weighted): 0.8221009325097248




Epoch 3:   0%|          | 0/581 [00:01<?, ?it/s, training_loss=0.078]

Epoch 3:   0%|          | 1/581 [00:01<13:51,  1.43s/it, training_loss=0.078]

Epoch 3:   0%|          | 1/581 [00:02<13:51,  1.43s/it, training_loss=0.042]

Epoch 3:   0%|          | 2/581 [00:02<13:51,  1.44s/it, training_loss=0.042]

Epoch 3:   0%|          | 2/581 [00:04<13:51,  1.44s/it, training_loss=0.039]

Epoch 3:   1%|          | 3/581 [00:04<13:50,  1.44s/it, training_loss=0.039]

Epoch 3:   1%|          | 3/581 [00:05<13:50,  1.44s/it, training_loss=0.021]

Epoch 3:   1%|          | 4/581 [00:05<13:48,  1.44s/it, training_loss=0.021]

Epoch 3:   1%|          | 4/581 [00:07<13:48,  1.44s/it, training_loss=0.046]

Epoch 3:   1%|          | 5/581 [00:07<13:47,  1.44s/it, training_loss=0.046]

Epoch 3:   1%|          | 5/581 [00:08<13:47,  1.44s/it, training_loss=0.126]

Epoch 3:   1%|          | 6/581 [00:08<13:45,  1.44s/it, training_loss=0.126]

Epoch 3:   1%|          | 6/581 [00:10<13:45,  1.44s/it, t


Epoch 3
Training loss: 0.25896473641213985



  0%|          | 0/3 [1:02:23<?, ?it/s]

100%|██████████| 3/3 [47:17<00:00, 945.96s/it]

Validation loss: 0.5755915859925378
F1 Score (Weighted): 0.8221009325097248


In [ ]:
accuracy_per_class(predictions, true_vals)

Class: Yes
Accuracy: 2647/2901

Class: No
Accuracy: 1848/2166

Class: In the middle, neither yes nor no
Accuracy: 37/127

Class: Probably yes / sometimes yes
Accuracy: 111/249

Class: Probably no
Accuracy: 48/232

Class: Yes, subject to some conditions
Accuracy: 466/517



In [ ]:
flat_accuracy(predictions, true_vals)

0.8328488372093024